In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from datetime import date, datetime, timedelta


In [2]:
cloud_config= {
  'secure_connect_bundle': r'C:/Users/rbbite/Desktop/cas.py/secure-connect-bigdata.zip'
}
auth_provider = PlainTextAuthProvider('AmRzfauuynnYJhzhBenNkfMo', '0Ytu_-4PR85_LI4Sahm_c_ErfvHiZ,M9DOu3PkmZ1+cQ33.XuTdb+GhH_ni,OwlC3XlUg-_Eh9GpODS19jCHERQpFMW1k3ts4pIBMRDWtIxsN5dMCx4oS7COwoHQhFHa') # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session2 = cluster.connect()
session3=cluster.connect()

#session.set_keyspace('keyspace1')

In [29]:
import pandas as pd


In [ ]:
import pandas as pd
import math

trips = pd.read_csv(r'C:\Users\rbbite\Desktop\cas.py\taxi_trip_data.csv', encoding= 'unicode_escape')  
trips



In [ ]:
session.execute("create type long_lat ( longitude float, latitude float );")


In [ ]:

trips['pickup_hour']=0
z=trips
z

In [9]:
p = {1: 'Credit', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown',6: 'Voided trip'}
z=z.replace({"payment_type": p})

In [28]:
session.execute("""
    create table if not exists airbnb.z(
     vendor_id int,
    pickup_datetime timestamp,
    dropoff_datetime timestamp,
    passenger_count int,
    trip_distance float,
    rate_code int,
    store_and_fwd_flag text,
    payment_type int,
    fare_amount float,
    extra float,
    mta_tax float,
    tip_amount float,
    tolls_amount float,
    imp_surcharge float,
    total_amount float,
    pickup_location_id int,
    dropoff_location_id int,
    pickup_hour text,
    Primary key (pickup_location_id,dropoff_location_id)
     );
""")

In [ ]:
# Reading the data
taxi_trip_data = pd.read_csv(r'C:\Users\rbbite\Desktop\cas.py\taxi_trip_data.csv', encoding= 'unicode_escape') 
taxi_zone_geo= pd.read_csv(r'C:\Users\rbbite\Desktop\cas.py\taxi_zone_geo.csv', encoding= 'unicode_escape')


In [ ]:
# Dropping “store_and_fwd_flag”, “rate_code” and “total_amount” columns from taxitripdata
taxi_trip_data.drop(['store_and_fwd_flag','rate_code','total_amount'], inplace=True, axis=1)
taxi_trip_data

In [6]:
# Checking if there are missing data
taxi_trip_data.isnull().values.any()

False

In [11]:
# Checking if there are missing data
taxi_zone_geo.isnull().values.any()

False

In [ ]:
#shahddd
# Getting the name column from the zone dataset into the taxi trip dataset:
taxi_trip_data = pd.merge(taxi_trip_data,taxi_zone_geo[['zone_id','zone_name']], left_on='pickup_location_id', right_on = 'zone_id')
#taxi_trip_data = taxi_trip_data.drop(['zone_id','zone_name_x','zone_name_y'], axis=1)


taxi_trip_data

In [11]:
# Taking a sample of 15000 as the data is huge and it takes a lot of time to be dealt with:
modified_taxi_trip_data=taxi_trip_data.sample(n=15000, replace=True)
modified_taxi_zone_geo=taxi_zone_geo.sample(n=15000, replace=True)

In [12]:
# Modifying the data type of certain columns and adding the one needed for the following queries:
modified_taxi_trip_data['dropoff_datetime']=pd.to_datetime(modified_taxi_trip_data['dropoff_datetime'])

modified_taxi_trip_data['pickup_datetime']=pd.to_datetime(modified_taxi_trip_data['pickup_datetime'])

modified_taxi_trip_data['duration']= 0 

modified_taxi_trip_data['trip_cost']= 0.0

modified_taxi_trip_data['type_of_day']= ''

In [13]:
# Modifyting the payment type to string:
p = {1: 'Credit', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown',6: 'Voided trip'}
modified_taxi_trip_data=modified_taxi_trip_data.replace({"payment_type": p})

In [14]:
# Dropping rows where passengers countis equal to 0 as they did not make a trip and their data won't be needed:
modified_taxi_trip_data = modified_taxi_trip_data[modified_taxi_trip_data['passenger_count'] >= 1]
modified_taxi_trip_data 

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,pickup_location_id,dropoff_location_id,zone_id,zone_name,duration,trip_cost,type_of_day
264681,1,2018-11-01 22:32:22,2018-11-01 22:40:59,1,0.90,Credit,7.0,0.5,0.5,1.65,0.0,0.3,48,170,48,Clinton East,0,0.0,
5153844,2,2018-11-14 18:19:05,2018-11-14 19:16:39,1,12.33,Credit,46.5,1.0,0.5,5.00,0.0,0.3,132,62,132,JFK Airport,0,0.0,
5262349,1,2018-03-14 19:08:08,2018-03-14 19:37:50,1,6.60,Cash,25.0,1.0,0.5,0.00,0.0,0.3,132,197,132,JFK Airport,0,0.0,
5807105,2,2018-06-02 00:31:15,2018-06-02 00:50:21,1,5.24,Cash,17.5,0.5,0.5,0.00,0.0,0.3,161,42,161,Midtown Center,0,0.0,
3834558,2,2018-06-27 02:01:00,2018-06-27 02:15:27,1,3.86,Credit,14.5,0.5,0.5,2.00,0.0,0.3,211,49,211,SoHo,0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4792067,2,2018-09-21 16:22:19,2018-09-21 16:35:22,4,1.10,Credit,9.5,1.0,0.5,2.26,0.0,0.3,237,229,237,Upper East Side South,0,0.0,
9341065,2,2018-03-14 07:52:23,2018-03-14 07:59:12,2,0.76,Credit,6.0,0.0,0.5,1.36,0.0,0.3,263,236,263,Yorkville West,0,0.0,
4497808,1,2018-03-05 19:26:15,2018-03-05 19:38:17,1,2.00,Credit,9.5,1.0,0.5,2.25,0.0,0.3,249,161,249,West Village,0,0.0,
5395795,2,2018-07-31 10:41:03,2018-07-31 10:53:01,2,2.34,Credit,11.0,0.0,0.5,2.36,0.0,0.3,125,246,125,Hudson Sq,0,0.0,


In [64]:
x= modified_taxi_trip_data.dtypes
y= modified_taxi_zone_geo.dtypes
print(x)
print(y)

vendor_id                int64
pickup_datetime         object
dropoff_datetime        object
passenger_count          int64
trip_distance          float64
payment_type            object
fare_amount            float64
extra                  float64
mta_tax                float64
tip_amount             float64
tolls_amount           float64
imp_surcharge          float64
pickup_location_id       int64
dropoff_location_id      int64
zone_id                  int64
zone_name               object
dtype: object
zone_id       int64
zone_name    object
borough      object
zone_geom    object
dtype: object


In [15]:
session.execute("""
     create table if not exists airbnb.tripss (
      vendor_id int,
         pickup_datetime timestamp,
         dropoff_datetime timestamp,
         passenger_count int, 
         trip_distance float,
         payment_type text,
         fare_amount float,
         extra float,
         mta_tax float ,
         tip_amount float,
         tolls_amount float,
         imp_surcharge float,
         pickup_location_id int ,
         dropoff_location_id int ,
         duration int,
         trip_cost float,
         type_of_day text,
         Primary key (pickup_location_id,dropoff_location_id)
  	);
 """)

In [16]:
rows,cols= modified_taxi_trip_data.shape
rows


14880

In [17]:
for x in range(0, 1000):
    session.execute("""
     INSERT INTO airbnb.tripss (
          vendor_id,
          pickup_datetime,
          dropoff_datetime,
          passenger_count,
          trip_distance,
          payment_type,
          fare_amount,
          extra,
          mta_tax,
          tip_amount,
          tolls_amount,
          imp_surcharge,
          pickup_location_id,
          dropoff_location_id,
          duration,
          trip_cost,
          type_of_day
     ) VALUES (
          %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
          %s, %s, %s, %s, %s, %s, %s
     );""",
     (
          int(modified_taxi_trip_data.iloc[x].vendor_id),
          str(modified_taxi_trip_data.iloc[x].pickup_datetime),
          str(modified_taxi_trip_data.iloc[x].dropoff_datetime),
          int(modified_taxi_trip_data.iloc[x].passenger_count),
          float(modified_taxi_trip_data.iloc[x].trip_distance),
          str(modified_taxi_trip_data.iloc[x].payment_type),
          float(modified_taxi_trip_data.iloc[x].fare_amount),
          float(modified_taxi_trip_data.iloc[x].extra),
          float(modified_taxi_trip_data.iloc[x].mta_tax),
          float(modified_taxi_trip_data.iloc[x].tip_amount),
          float(modified_taxi_trip_data.iloc[x].tolls_amount),
          float(modified_taxi_trip_data.iloc[x].imp_surcharge),
          int(modified_taxi_trip_data.iloc[x].pickup_location_id),
          int(modified_taxi_trip_data.iloc[x].dropoff_location_id),
          int(modified_taxi_trip_data.iloc[x].duration),
          float(modified_taxi_trip_data.iloc[x].trip_cost),
          str(modified_taxi_trip_data.iloc[x].type_of_day)
     )
    )

In [18]:
 # Creating table in my casandra database:
session3.execute("""
     CREATE TABLE IF NOT EXISTS timetrip.trip (
         vendor_id INT,
         pickup_datetime TIMESTAMP,
         dropoff_datetime TIMESTAMP,
         passenger_count INT,
         trip_distance FLOAT,
         payment_type TEXT,
         fare_amount FLOAT,
         extra FLOAT,
         mta_tax FLOAT ,
         tip_amount FLOAT,
         tolls_amount FLOAT,
         imp_surcharge FLOAT,
         pickup_location_id INT ,
         dropoff_location_id INT ,
         duration INT,
         trip_cost FLOAT,
         type_of_day TEXT,
         zone_name Text,
         PRIMARY KEY (pickup_location_id, dropoff_location_id)
  	);
 """)


In [17]:
rows,cols= modified_taxi_trip_data.shape
rows

14862

In [ ]:
for x in range(0, rows):
    session3.execute("""
     INSERT INTO timetrip.trip(
           vendor_id,
          pickup_datetime,
          dropoff_datetime,
          passenger_count,
          trip_distance,
          payment_type,
          fare_amount,
          extra,
          mta_tax,
          tip_amount,
          tolls_amount,
          imp_surcharge,
          pickup_location_id,
          dropoff_location_id,
          duration,
          trip_cost,
          type_of_day,
          zone_name
     ) VALUES (
          %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
          %s, %s, %s, %s, %s, %s, %s,%s
     );""",
     (
          int(modified_taxi_trip_data.iloc[x].vendor_id),
          str(modified_taxi_trip_data.iloc[x].pickup_datetime),
          str(modified_taxi_trip_data.iloc[x].dropoff_datetime),
          int(modified_taxi_trip_data.iloc[x].passenger_count),
          float(modified_taxi_trip_data.iloc[x].trip_distance),
          str(modified_taxi_trip_data.iloc[x].payment_type),
          float(modified_taxi_trip_data.iloc[x].fare_amount),
          float(modified_taxi_trip_data.iloc[x].extra),
          float(modified_taxi_trip_data.iloc[x].mta_tax),
          float(modified_taxi_trip_data.iloc[x].tip_amount),
          float(modified_taxi_trip_data.iloc[x].tolls_amount),
          float(modified_taxi_trip_data.iloc[x].imp_surcharge),
          int(modified_taxi_trip_data.iloc[x].pickup_location_id),
          int(modified_taxi_trip_data.iloc[x].dropoff_location_id),
          int(modified_taxi_trip_data.iloc[x].duration),
          float(modified_taxi_trip_data.iloc[x].trip_cost),
          str(modified_taxi_trip_data.iloc[x].type_of_day),
          str(modified_taxi_trip_data.iloc[x].zone_name)

     )
)

In [12]:
value = session.execute("select pickup_datetime , dropoff_datetime from airbnb.tripss")
duration=[]
for i in value: 
    pickup_datetime= i[0]
    dropoff_datetime=i[1]
    d=dropoff_datetime-pickup_datetime
    d=d.seconds/60 
    duration.append(d)


In [21]:
#d)Calculate the duration and add it to the field which is called duration

firstpart=session.execute("SELECT dropoff_datetime, pickup_datetime,pickup_location_id,dropoff_location_id FROM airbnb.tripss")
for row in firstpart:
    dropoff_datetime = row.dropoff_datetime
    pickup_datetime = row.pickup_datetime
    pickup_location_id=row.pickup_location_id
    dropoff_location_id=row.dropoff_location_id
    duration = (dropoff_datetime - pickup_datetime).seconds/60
    duration_int=int(duration) 
    session.execute("INSERT INTO airbnb.tripss (duration,pickup_location_id,dropoff_location_id) VALUES (%s, %s,%s)", (duration_int,pickup_location_id,dropoff_location_id))




In [29]:
value=session.execute("select *from airbnb.tripss limit 5")
i=0
for row in value:
    print(row)

Row(pickup_location_id=114, dropoff_location_id=36, dropoff_datetime=datetime.datetime(2018, 8, 19, 0, 53, 6), duration=32, extra=0.5, fare_amount=23.5, imp_surcharge=0.30000001192092896, mta_tax=0.5, passenger_count=3, payment_type='Credit', pickup_datetime=datetime.datetime(2018, 8, 19, 0, 21, 6), tip_amount=4.960000038146973, tolls_amount=0.0, trip_cost=29.760000228881836, trip_distance=5.559999942779541, type_of_day='', vendor_id=2)
Row(pickup_location_id=114, dropoff_location_id=37, dropoff_datetime=datetime.datetime(2018, 2, 19, 15, 3, 57), duration=20, extra=0.0, fare_amount=17.5, imp_surcharge=0.30000001192092896, mta_tax=0.5, passenger_count=1, payment_type='Credit', pickup_datetime=datetime.datetime(2018, 2, 19, 14, 43, 41), tip_amount=3.6600000858306885, tolls_amount=0.0, trip_cost=0.0, trip_distance=4.449999809265137, type_of_day='', vendor_id=2)
Row(pickup_location_id=114, dropoff_location_id=79, dropoff_datetime=datetime.datetime(2018, 7, 22, 17, 50, 52), duration=7, extr

In [85]:
#e) Use "fare amount"."extra" ,"mta_tax", "tip amount","tolls amount" and "imp_ surcharge" to calculate the total trip cost and add it as a new field in your database
total=session.execute("SELECT fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,pickup_location_id,dropoff_location_id FROM airbnb.tripss")
for row in total:
     fare_amount = row.fare_amount
     extra=row.extra
     mta_tax=row.mta_tax
     tip_amount=row.tip_amount
     tolls_amount=row.tolls_amount
     imp_surcharge=row.imp_surcharge
     pickup_location_id=row.pickup_location_id
     dropoff_location_id=row.dropoff_location_id

     sum=fare_amount+extra+mta_tax+tip_amount+tolls_amount+imp_surcharge
     session.execute("INSERT INTO airbnb.tripss (trip_cost,pickup_location_id,dropoff_location_id) VALUES (%s,%s,%s)", (sum,pickup_location_id,dropoff_location_id))
     


In [86]:
value=session.execute("select *from airbnb.tripss limit 5")
i=0
for row in value:
    print(row)

Row(pickup_location_id=114, dropoff_location_id=36, dropoff_datetime=datetime.datetime(2018, 8, 19, 0, 53, 6), duration=32, extra=0.5, fare_amount=23.5, imp_surcharge=0.30000001192092896, mta_tax=0.5, passenger_count=3, payment_type='Credit', pickup_datetime=datetime.datetime(2018, 8, 19, 0, 21, 6), tip_amount=4.960000038146973, tolls_amount=0.0, trip_cost=29.760000228881836, trip_distance=5.559999942779541, type_of_day='', vendor_id=2)
Row(pickup_location_id=114, dropoff_location_id=90, dropoff_datetime=datetime.datetime(2018, 7, 15, 10, 47, 55), duration=4, extra=0.0, fare_amount=6.5, imp_surcharge=0.30000001192092896, mta_tax=0.5, passenger_count=1, payment_type='Credit', pickup_datetime=datetime.datetime(2018, 7, 15, 10, 43, 5), tip_amount=1.4500000476837158, tolls_amount=0.0, trip_cost=8.75, trip_distance=1.5, type_of_day='', vendor_id=1)
Row(pickup_location_id=114, dropoff_location_id=107, dropoff_datetime=datetime.datetime(2018, 6, 2, 3, 8, 21), duration=8, extra=0.5, fare_amoun

In [26]:

#f)What is the most common payment type used per time of day
from collections import Counter
from datetime import datetime
from collections import Counter

time=[]
Morning=[]
Afternoon=[]
Evening=[]
Night=[]
type=[]
result = session.execute("SELECT pickup_datetime,payment_type FROM airbnb.tripss")
for row in result:
    pickup_datetime = row.pickup_datetime
    payment_type=row.payment_type
    pickup_datetime_str = pickup_datetime.strftime("%Y-%m-%d %H:%M:%S")
    pickup_hour = datetime.strptime(pickup_datetime_str, "%Y-%m-%d %H:%M:%S").hour
    time.append(pickup_hour) 
    type.append(payment_type)
k=0 
for i in time:
    if (5<=time[k] and time[k]<12 ):
         Morning.append(type[k])
    elif (12<=time[k] and time[k]<17 ):
        Afternoon.append(type[k])
    elif (17<=time[k] and time[k]<21 ):
        Evening.append(type[k])
    else:
        Night.append(type[k])
    k = k +1  

def most_repeated(List):
    counter = 0
    num = List[0]
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
    return num
REPEATED_MORNING= most_repeated(Morning)
REPEATED_AFTERNOON= most_repeated(Afternoon)
REPEATED_EVENING= most_repeated(Evening)
REPEATED_NIGHT= most_repeated(Night) 



print("MOST REPEATED PAYMENT IN MORNING IS"+  REPEATED_MORNING )
print("MOST REPEATED PAYMENT IN AFTERNOON IS"+ REPEATED_AFTERNOON)
print("MOST REPEATED PAYMENT IN EVENING IS"+ REPEATED_EVENING)
print("MOST REPEATED PAYMENT IN NIGHT IS"+ REPEATED_NIGHT)





MOST REPEATED PAYMENT IN MORNING ISCredit
MOST REPEATED PAYMENT IN AFTERNOON ISCredit
MOST REPEATED PAYMENT IN EVENING ISCredit
MOST REPEATED PAYMENT IN NIGHT ISCredit


In [ ]:
#f) in casandra
#BAHAWEL AHAWEL B CASANDRA L HOUR  AMLENLY ERRORS fa hastakhdem eltwo lines elestakhdemtohom foo2  fa hatta mesh arfa akamelha b cassandra

#result = session.execute("SELECT dateOf(toTimestamp(pickup_datetime)) as hour_column  FROM airbnb.tripss")
#resultttt = session3.execute("SELECT hour(pickup_datetime) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT substring(pickup_datetime from 12 for 2) as pickup_datetime_hour FROM `timetrip.trip`")
#resultttt = session3.execute("SELECT hour(dateOf(pickup_datetime)) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(toTimestamp(toUnixTimestamp(pickup_datetime))) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(toTimestamp(toDate(pickup_datetime))) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT blobAsInt(dateAsBlob(dateOf(pickup_datetime)), 0, 2) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(dateOf(toTimestamp(toUnixTimestamp(pickup_datetime)))) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(dateOf(toTimestamp(toDate(fromUnixtime(pickup_datetime)))))) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(dateOf(toTimestamp(toDate(fromUnixtime(pickup_datetime))))) as pickup_datetime_hour FROM timetrip.trip;")
#resultttt = session3.execute("SELECT hour(dateOf(toTimestamp(pickup_datetime/1000000))) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(dateOf(toTimestamp(pickup_datetime/bigint(1000000)))) as pickup_datetime_hour FROM timetrip.trip")
#resultttt = session3.execute("SELECT hour(dateOf(toTimestamp(pickup_datetime/CAST(1000000 AS BIGINT)))) as pickup_datetime_hour FROM timetrip.trip")

#hena ana ha3teber eno eshtaghal w eny ma3aya elhour f column esmo pickup_datetime_hour 
 
time=[]
Morning=[]
Afternoon=[]
Evening=[]
Night=[]
type=[] 

for rows in resultttt:
 pickup_datetime_hour=rows.pickup_datetime_hour
 query1 = ("SELECT pickup_datetime FROM airbnb.tripss WHERE pickup_datetime <=5 AND pickup_datetime <=12" )  
 query2 = ("SELECT pickup_datetime FROM airbnb.tripss WHERE pickup_datetime <=12 AND pickup_datetime <=17" )  
 query3 = ("SELECT pickup_datetime FROM airbnb.tripss WHERE pickup_datetime <=17 AND pickup_datetime <=21" ) 

for i in time:
 if(query1):
     Morning.append(type[k])
 elif(query2):
     Afternoon.append(type[k])
 elif(query2):
     Evening.append(type[k])
 else: 
     Night.append(type[k])
k = k +1  

session3.execute("""CREATE TEMPORARY TABLE list (
    value INT
);""") 
#di hastakhdemha badal elfunction eltanyaa beta3et most repeated 
session3.execute("""INSERT INTO list (value) VALUES (Morning),(Afternoon), (Evening), (Night)""") #henaa hahot elvalues elana 3ayzaha

query=("SELECT value, COUNT(*) AS frequency FROM list GROUP BY value ORDER BY frequency DESC LIMIT 1")
   

In [48]:
time=[]
result = session.execute("SELECT pickup_datetime, pickup_location_id, dropoff_location_id,pickup_hour FROM airbnb.z")

for row in result:
    pickup_hour=row.pickup_hour
    pickup_location_id = row.pickup_location_id
    dropoff_location_id = row.dropoff_location_id
    pickup_datetime = row.pickup_datetime

    if pickup_datetime is not None:
        pickup_datetime_str = pickup_datetime.strftime("%Y-%m-%d %H:%M:%S")
        pickup_hourr = datetime.strptime(pickup_datetime_str, "%Y-%m-%d %H:%M:%S").hour
        time.append(pickup_hourr)
    else:
        time.append(None)

k=0
if (time[k] is not None  and time[k]<12 and  time[k]>5):
       session.execute("INSERT INTO airbnb.z( pickup_location_id,dropoff_location_id,pickup_hour) VALUES ( %s,%s,%s)", (pickup_location_id ,dropoff_location_id ,"Morning"))
elif( time[k] is not None and time[k]> 12 and time[k]<17):
      session.execute("INSERT INTO airbnb.z( pickup_location_id,dropoff_location_id,pickup_hour) VALUES ( %s,%s,%s)", (pickup_location_id ,dropoff_location_id ,"Afternoon"))
#else:      
      #session.execute("INSERT INTO airbnb.z( pickup_location_id,dropoff_location_id,pickup_hour) VALUES ( %s,%s,%s)", (pickup_location_id ,dropoff_location_id ,"Night"))
k=k+1

In [ ]:
#f) tany mohawlaa en a3melhaa b elcolumn elgedeed elna 3amalto
    #w bardoo betala3ly errorr khalasss erefttt 
timee=[]
Morning=[]
Afternoon=[]
Night=[]
type=[] 


query1 = session.execute("SELECT payment_type FROM airbnb.z WHERE pickup_hour='Morning'")  
query2 = session.execute("SELECT payment_type FROM airbnb.z WHERE pickup_hour='Afternoon'" )  
#query3 = session.execute("SELECT payment_type FROM airbnb.z WHERE pickup_hour 'Night'" )  



for row in query1:
  Morning.append(row)
for row in query2:
  Afternoon.append(row)

print(Morning)



In [45]:
#number of unique numbers of passengers
print(modified_taxi_trip_data['passenger_count'].unique())
#do we have from 1 to 6 unique values 

[1 4 5 2 3 6]


In [32]:
#g)What is the average tip amount per passenger count b pythonn
  #ana hena ayza l kol passenger count elaverage tip amount beta3o

result1 = session.execute("SELECT AVG(tip_amount) as average FROM airbnb.tripss WHERE passenger_count = 1 ALLOW FILTERING")
for row in result1:
    tip_amount_for_1_passenger=row.average

result2 = session.execute("SELECT AVG(tip_amount) as average FROM airbnb.tripss WHERE passenger_count = 2 ALLOW FILTERING")
for row in result2:
    tip_amount_for_2_passenger=row.average

result3 = session.execute("SELECT AVG(tip_amount) as average FROM airbnb.tripss WHERE passenger_count = 3 ALLOW FILTERING")
for row in result3:
    tip_amount_for_3_passenger=row.average

result4 = session.execute("SELECT AVG(tip_amount) as average FROM airbnb.tripss WHERE passenger_count = 4 ALLOW FILTERING")
for row in result4:
    tip_amount_for_4_passenger=row.average

result5 = session.execute("SELECT AVG(tip_amount) as average FROM airbnb.tripss WHERE passenger_count = 5 ALLOW FILTERING")
for row in result5:
    tip_amount_for_5_passenger=row.average

result6 = session.execute("SELECT AVG(tip_amount) as average FROM airbnb.tripss WHERE passenger_count = 6 ALLOW FILTERING")
for row in result6:
    tip_amount_for_6_passenger=row.average

print("Average tip amount for 1 passenger: ${:.2f}".format(tip_amount_for_1_passenger))
print("Average tip amount for 2 passengers: ${:.2f}".format(tip_amount_for_2_passenger))
print("Average tip amount for 3 passengers: ${:.2f}".format(tip_amount_for_3_passenger))
print("Average tip amount for 4 passengers: ${:.2f}".format(tip_amount_for_4_passenger))
print("Average tip amount for 5 passengers: ${:.2f}".format(tip_amount_for_5_passenger))
print("Average tip amount for 6 passengers: ${:.2f}".format(tip_amount_for_6_passenger))


Average tip amount for 1 passenger: $2.26
Average tip amount for 2 passengers: $2.18
Average tip amount for 3 passengers: $1.92
Average tip amount for 4 passengers: $1.09
Average tip amount for 5 passengers: $1.81
Average tip amount for 6 passengers: $1.74
